### Summary
Record mean 1 row of the csv file. 

This notebook converts the records into alarms. 

In [1]:
import pandas as pd
import gc # for forcing garbage collection
import time
from alarms import  convertRecordsToAlarmsV1
from alarms import  convertRecordsToAlarmsV2
start = time.time()

In [2]:
files = ["haziran2019.csv","march2019.csv","mayis2019.csv","nisan2019.csv"]
files = ["formatted-pre-1-"+f for f in files]
cols = ["MachineName","SourceName","EventTime", "Message","MessageType","Quality","Condition","Mask","NewState","Status"]
path = "./data/new/"

In [3]:
# for f in files:    
#     print("==================== File : {} =============".format(f))
#     input_fname = f
#     output_fname = "formatted-pre-2-" + input_fname.split("-")[-1]
#     df = pd.read_csv(path + input_fname, low_memory=False, usecols=cols ,parse_dates=["EventTime"])
#     print(df.info())
#     alarms = []
#     print("Total Number of Sources :  ", len(df["SourceName"].unique()))
#     print("Staring Processing records of each source : ", end= " ")
#     index = 0 # only for printing 
#     for sname in df["SourceName"].unique():
#         records =  df.loc[df['SourceName'].isin([sname])]
#         records = records.to_dict(orient="records")
#         temp_alarms = convertRecordsToAlarms(records)
#         print("({},{},{})".format(index,sname, len(temp_alarms)), end=" ")
#         alarms += temp_alarms # concat list
#         index += 1
        
#     print("\n\n Done: All records are converted to to alarms.")
#     # writing to files
#     df2 = pd.DataFrame(alarms)
#     df2.to_csv(path + output_fname, index = False)
#     print("Processing Time = {} minutes".format((time.time()-start)/60.0))
#     print(df2)

# print(">Done: Total Time to Complete : ", time.time() - start)

In [4]:
df = pd.read_csv(path + "formatted-pre-1-mayis2019.csv", low_memory=False, usecols=cols ,parse_dates=["EventTime"])
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1776439 entries, 0 to 1776438
Data columns (total 10 columns):
 #   Column       Dtype         
---  ------       -----         
 0   MachineName  object        
 1   SourceName   object        
 2   EventTime    datetime64[ns]
 3   Message      object        
 4   Quality      int64         
 5   Condition    object        
 6   Mask         int64         
 7   NewState     int64         
 8   Status       int64         
 9   MessageType  object        
dtypes: datetime64[ns](1), int64(4), object(5)
memory usage: 135.5+ MB
None


In [5]:
df["MessageType"].describe()

count        1776439
unique             3
top       Activation
freq          902467
Name: MessageType, dtype: object

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1776439 entries, 0 to 1776438
Data columns (total 10 columns):
 #   Column       Dtype         
---  ------       -----         
 0   MachineName  object        
 1   SourceName   object        
 2   EventTime    datetime64[ns]
 3   Message      object        
 4   Quality      int64         
 5   Condition    object        
 6   Mask         int64         
 7   NewState     int64         
 8   Status       int64         
 9   MessageType  object        
dtypes: datetime64[ns](1), int64(4), object(5)
memory usage: 135.5+ MB


In [7]:



# df_of_top_x_sensors = df_csv.loc[df_csv['SourceName'].isin(temp_s.keys()[:num_top_sensors])]

In [8]:
alarms = []
differs = []
sources_ranks_dict = df['SourceName'].value_counts()
for sname in sources_ranks_dict.keys():
    df_sname = df.loc[df['SourceName'].isin([sname])] # source DF
    print(">> Source: {}, Conditions: {}".format(sname, df_sname["Condition"].unique()), end="=>")
    types_rank_dict = df_sname["MessageType"].value_counts() # source ranks
    total = 0
    for key in types_rank_dict.keys():
        print(key, types_rank_dict[key], end=",")
        total += types_rank_dict[key]
    # if len(types_rank_dict.keys()) > 2:
    #     print(">> Source: {}".format(sname, sources_ranks_dict[sname]), end="=>")
    #     print(types_rank_dict, end="")

    stime = time.time()
    temp_alarms1 = convertRecordsToAlarmsV1(df_sname)
    # print("Time1", time.time()-stime, end=",")
    alarms += temp_alarms1
    stime = time.time()
    temp_alarms2 = convertRecordsToAlarmsV2(df_sname.to_dict(orient="records"))
    # print("Time2", time.time()-stime, end=",")
    print("ALARMS1: {}, Alarms2: {}".format(len(temp_alarms1),len(temp_alarms2)),end="")
    assert(total== sources_ranks_dict[sname]) # sum is equal to count 
    # assert(len(temp_alarms1) == len(temp_alarms2))  
    if (len(temp_alarms1) != len(temp_alarms2)):
        differs.append(sname)
    print("")

    # writing to files
df2 = pd.DataFrame(alarms)
df2.to_csv(path + "formatted-pre-2-mayis2019.csv", index = False)
# print("Processing Time = {} minutes".format((time.time()-start)/60.0))
df2

>> Source: 48TIC2026, Conditions: ['VEL+' 'VEL-']=>Activation 542322,Recover 531132,ALARMS1: 530167, Alarms2: 531132
>> Source: 47TI931A, Conditions: ['IOP']=>Activation 241468,Recover 235835,ALARMS1: 235712, Alarms2: 235834
>> Source: 19LI105A, Conditions: ['LO' 'VEL-' 'LL' 'VEL+']=>Activation 15153,Recover 15151,ALARMS1: 15109, Alarms2: 15147
>> Source: 48TIC2027, Conditions: ['IOP-']=>Activation 12942,Recover 12773,ALARMS1: 12754, Alarms2: 12771
>> Source: 48FIX2040, Conditions: ['IOP-']=>Activation 12942,Recover 12773,ALARMS1: 12754, Alarms2: 12771
>> Source: 47TI002C, Conditions: ['LLL' 'LTRP']=>Activation 7728,Recover 7709,ALARMS1: 6224, Alarms2: 7706
>> Source: 47TI003B, Conditions: ['LTRP' 'LLL']=>Activation 7149,Recover 7139,ALARMS1: 5994, Alarms2: 7136
>> Source: 47FI1575, Conditions: ['IOP' 'VEL+' 'VEL-']=>Activation 5268,Recover 4957,ALARMS1: 4956, Alarms2: 4956
>> Source: 48PI2025, Conditions: ['IOP' 'IOP-' 'HH' 'HI']=>Activation 5039,Recover 4741,ALARMS1: 4733, Alarms2: 4

,MachineName,SourceName,Message,Quality,Condition,Mask,NewState,Status,MessageType,StartTime,EndTime,EndMessage
0,PHD47B,48TIC2026,48TIC2026 E208 CKS TEMIZ MDEA PV = -4.8 C VEL+,192,VEL+,193,3,0,Activation,2019-05-01 00:00:02,2019-05-01 00:00:03,48TIC2026 E208 CKS TEMIZ MDEA PV = -4.9 C VEL+...
1,PHD47B,48TIC2026,48TIC2026 E208 CKS TEMIZ MDEA PV = -4.8 C VEL+,192,VEL+,193,3,0,Activation,2019-05-01 00:00:10,2019-05-01 00:00:12,48TIC2026 E208 CKS TEMIZ MDEA PV = -5.3 C VEL+...
2,PHD47B,48TIC2026,48TIC2026 E208 CKS TEMIZ MDEA PV = -5.0 C VEL+,192,VEL+,193,3,0,Activation,2019-05-01 00:00:30,2019-05-01 00:00:32,48TIC2026 E208 CKS TEMIZ MDEA PV = -5.0 C VEL+...
3,PHD47B,48TIC2026,48TIC2026 E208 CKS TEMIZ MDEA PV = -4.8 C VEL+,192,VEL+,193,3,0,Activation,2019-05-01 00:00:39,2019-05-01 00:00:41,48TIC2026 E208 CKS TEMIZ MDEA PV = -5.1 C VEL+...
4,PHD47B,48TIC2026,48TIC2026 E208 CKS TEMIZ MDEA PV = -5.0 C VEL+,192,VEL+,193,3,0,Activation,2019-05-01 00:00:47,2019-05-01 00:00:49,48TIC2026 E208 CKS TEMIZ MDEA PV = -5.3 C VEL+...
...,...,...,...,...,...,...,...,...,...,...,...,...
868720,PHD47B,47PI851C,47PI851C 47G2A YAGLAMA YAGI PRS C PV = 1.17 Kg...,192,LLL,241,3,0,Activation,2019-05-13 14:29:27,2019-05-13 14:29:28,47PI851C 47G2A YAGLAMA YAGI PRS C PV = 1.40 Kg...
868721,PHD47B,47TI3415,47TI3415 C303 DIP CKSTEMIZ DEA ACK,192,LO,194,7,0,Activation,2019-05-13 16:33:28,2019-05-15 06:02:07,47TI3415 C303 DIP CKSTEMIZ DEA PV = 122 C LO R...
868722,PHD47B,47PI851ABC,47PI851ABC 47G2A YAGLM YAGI PRS ABC PV = 1.17 ...,192,LLL,241,3,0,Activation,2019-05-13 14:29:27,2019-05-13 14:29:28,47PI851ABC 47G2A YAGLM YAGI PRS ABC PV = 1.40 ...
868723,PHD47B,47TI1717,47TI1717 47E-130 ÃIKIS KERO SIC. ACK,192,LO,194,7,0,Activation,2019-05-13 15:07:00,2019-05-15 11:08:42,47TI1717 47E-130 ÃIKIS KERO SIC. PV = 130.1 C...


In [10]:
print(differs,len(differs))
sname = "47TAL002CD-ANN"
df_source = df.loc[df['SourceName'].isin([sname])]

alarms1 = convertRecordsToAlarmsV1(df_source)
alarms2 = convertRecordsToAlarmsV2(df_source.to_dict(orient="records"))
len(alarms1), len(alarms2)

['48TIC2026', '47TI931A', '19LI105A', '48TIC2027', '48FIX2040', '47TI002C', '47TI003B', '48PI2025', '47TI1713', '47TI003A', '47FI1568', '47PDAH605-ANN', '47TAL003AB-ANN', '48AAHH004-ANN', '48AI004', '47FI1003', '48XL001-ANN', '47TIC3520', '47AI2003', 'SIO-18XA001', '47TI1769', '47PIC1505', '47PIC2033A', '47TAL002CD-ANN', '47FI1005', '48BAL007-ANN', '48UA007-ANN', '47TAH808-ANN', '48HA019-ANN', '47TI2112A', '47TI3519', '47TI002D', '48LI013', '47TI002B', '47FI3016', '47TAL002AB-ANN', '47TI1506', '47PIC2028', '47TI1512ABC', '47LI850C', '47TI3531', '47PIC2023', '48XL006-ANN', '47LI1550ABC', '47TI808', '47PAL1508-ANN', '47LI850A', '48TI035B', '47XL011-ANN', '47PI2148ABC', '47TI753D-2'] 51


(109, 126)